In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib as plt

In [2]:
data=pd.read_csv("Score.csv")
data

,Unnamed: 0,Index,Sex,CHI611,MATH611,ENG611,CHI612,MATH612,ENG612,CHI621,...,CHI912,MATH912,ENG912,PHY912,CHE912,CHI921,MATH921,ENG921,PHY921,CHE921
0,1,A01,F,75.0,83.0,95.0,77.0,95.0,96.0,89.0,...,130.0,134.0,136.5,88.0,94.0,120.0,137.0,140.0,79.0,57.0
1,2,A02,F,88.0,100.0,90.0,88.0,100.0,95.0,91.0,...,132.0,146.0,142.0,97.0,96.0,134.0,138.0,145.5,84.0,57.0
2,3,A03,F,85.0,97.0,91.0,87.0,96.0,97.0,93.0,...,132.0,146.0,144.5,96.0,95.0,134.0,145.0,146.0,86.0,57.0
3,4,A04,F,83.0,97.0,97.0,83.0,94.0,97.0,86.0,...,133.0,129.0,142.0,84.0,94.0,121.0,136.0,140.0,83.0,53.0
4,5,A05,F,81.0,89.0,94.0,84.0,89.0,90.0,86.0,...,127.0,137.0,136.0,87.0,92.0,124.0,140.0,128.0,79.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,239,G26,M,41.0,43.0,57.0,51.0,63.0,69.0,69.0,...,69.0,54.0,65.0,57.0,74.0,54.0,68.0,74.0,41.0,42.0
239,240,G27,M,70.0,80.0,77.0,69.0,75.0,77.0,76.0,...,119.0,133.0,123.0,88.0,86.0,112.0,114.0,134.5,74.0,52.0
240,241,G28,M,69.0,81.0,53.0,62.0,69.0,50.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241,242,G29,M,59.0,12.0,62.0,57.0,9.0,54.0,48.0,...,89.0,10.0,58.0,30.0,25.0,78.0,26.0,77.5,29.0,9.0


In [3]:
col=np.array(data.columns)
col=np.delete(col,range(3))
col

array(['CHI611', 'MATH611', 'ENG611', 'CHI612', 'MATH612', 'ENG612',
       'CHI621', 'MATH621', 'ENG621', 'CHI622', 'MATH622', 'ENG622',
       'CHI711', 'MATH711', 'ENG711', 'CHI712', 'MATH712', 'ENG712',
       'CHI721', 'MATH721', 'ENG721', 'CHI722', 'MATH722', 'ENG722',
       'CHI811', 'MATH811', 'ENG811', 'PHY811', 'CHI812', 'MATH812',
       'ENG812', 'PHY812', 'CHI821', 'MATH821', 'ENG821', 'PHY821',
       'CHI822', 'MATH822', 'ENG822', 'PHY822', 'CHI911', 'MATH911',
       'ENG911', 'PHY911', 'CHE911', 'CHI912', 'MATH912', 'ENG912',
       'PHY912', 'CHE912', 'CHI921', 'MATH921', 'ENG921', 'PHY921',
       'CHE921'], dtype=object)

In [4]:
'''
maxScore1=[100 for i in range(36)]
maxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]
maxScore=maxScore1+maxScore2
'''

'\nmaxScore1=[100 for i in range(36)]\nmaxScore2=[150,150,150,100,150,150,150,100,60,150,150,150,100,100,150,150,150,90,60]\nmaxScore=maxScore1+maxScore2\n'

In [5]:
transDataDict={}
for i in range(55):
    temp=(data[col[i]]-np.mean(data[col[i]]))/(np.max(data[col[i]])-np.min(data[col[i]]))
    transDataDict[col[i]]=temp
transData=pd.DataFrame.from_dict(transDataDict)
del transDataDict

In [6]:
mergeData=transData.apply(lambda x:[x['CHI611'],x['CHI612'],x['CHI621'],x['CHI622'],
x['CHI711'],x['CHI712'],x['CHI721'],x['CHI722'],
x['CHI811'],x['CHI812'],x['CHI821'],x['CHI822'],
x['CHI911'],x['CHI912'],x['CHI921']],axis=1)
mergeData=pd.DataFrame(mergeData,columns=['CHI'])
mergeData['CHINoNan']=mergeData['CHI'].apply(lambda x:[i for i in x if not np.isnan(i)])
mergeData['CHILen']=mergeData['CHINoNan'].apply(lambda x:len(x))

In [7]:
def merge(x,col):
    if col in ['MATH','ENG']:
        x[col]=transData.apply(lambda x:[x[col+'611'],x[col+'612'],x[col+'621'],x[col+'622'],
        x[col+'711'],x[col+'712'],x[col+'721'],x[col+'722'],
        x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'PHY':
        x[col]=transData.apply(lambda x:[x[col+'811'],x[col+'812'],x[col+'821'],x[col+'822'],
        x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
    elif col == 'CHE':
        x[col]=transData.apply(lambda x:[x[col+'911'],x[col+'912'],x[col+'921']],axis=1)
def NoNan(x,col):
    x[col+'NoNan']=x[col].apply(lambda x:[i for i in x if not np.isnan(i)])
    x[col+'Len']=x[col+'NoNan'].apply(lambda x:len(x))
def delOrigin(x,col):
    x.drop([col],axis=1,inplace=True)

delOrigin(mergeData,'CHI')

for i in ['MATH','ENG','PHY','CHE']:
    merge(mergeData,i)
    NoNan(mergeData,i)
    delOrigin(mergeData,i)

In [8]:
def normTest(x,col):
    if x[col+'Len']>=3:
        return stats.shapiro(x[col+'NoNan'])
    else:
        return (-1,-1)

for i in ['CHI','MATH','ENG']:
    mergeData[i+'Norm']=mergeData.apply(lambda x:normTest(x,i),axis=1)

In [9]:
def stdTest(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    return stats.levene(temp[0],temp[1],temp[2])
mergeData['stdTest']=mergeData.apply(stdTest,axis=1)

In [10]:
equalStdData=mergeData[mergeData['stdTest'].apply(lambda x:x[1]>0.05)].copy()

In [11]:
def oneWay(df):
    temp=df[['CHINoNan','MATHNoNan','ENGNoNan']].values
    return stats.f_oneway(temp[0],temp[1],temp[2])
equalStdData['oneWay']=equalStdData.apply(oneWay,axis=1)

In [12]:
len(equalStdData[equalStdData['oneWay'].apply(lambda x:x[1]>0.05)])

54

In [13]:
equalStdData['isBias']=equalStdData['oneWay'].apply(lambda x:x[1]<0.05)

In [14]:
sexBiasCount=equalStdData.join(data['Sex'])
sexBiasCount=sexBiasCount.groupby(['Sex','isBias'])['CHINoNan'].count().reset_index()
sexBiasCount.rename(columns={'CHINoNan':'count'},inplace=True)
sexBiasCount=sexBiasCount.pivot(index='Sex',columns='isBias',values='count')

In [15]:
sexBiasCount

isBias,False,True
Sex,,
F,23,64
M,25,71


In [16]:
stats.chi2_contingency(sexBiasCount)

(0.011571704382183943,
 0.9143353224117214,
 1,
 array([[22.81967213, 64.18032787],
        [25.18032787, 70.81967213]]))

In [17]:
for i in ['CHI','MATH','ENG']:
    mergeData[i+'Mean']=mergeData[i+'NoNan'].apply(np.mean)

In [19]:
def findBias(df):
    meanCol=['CHIMean','MATHMean','ENGMean']
    dis=np.zeros(3)
    maxScoreLesson=np.argmax(df[meanCol])
    for i in range(3):
        temp=0
        for j in meanCol:
            temp+=np.abs(df[meanCol[i]]-df[j])
        dis[i]=temp
    argMax=np.argmax(dis)
    flag=(maxScoreLesson==argMax)
    return (np.max(dis),argMax,flag)
mergeData['bias']=mergeData.apply(findBias,axis=1)

In [20]:
mergeData['biasDis']=mergeData['bias'].apply(lambda x:x[0])
mergeData['biasClass']=mergeData['bias'].apply(lambda x:x[1])
mergeData['biasPostive']=mergeData['bias'].apply(lambda x:x[2])

In [23]:
biasData=equalStdData.join(mergeData[['biasDis','biasClass','biasPostive']])[['isBias','biasDis','biasClass','biasPostive']]

In [62]:
classPostiveData=biasData[biasData['isBias']>0].groupby(['biasClass','biasPostive'])['biasDis'].count().reset_index()
temp=classPostiveData.pivot(index='biasPostive',columns='biasClass',values='biasDis')

In [63]:
stats.chi2_contingency(temp)

(7.913342173027404,
 0.01912667949269471,
 2,
 array([[31.76923077, 22.61538462, 22.61538462],
        [27.23076923, 19.38461538, 19.38461538]]))

In [64]:
temp

biasClass,0,1,2
biasPostive,,,
False,40,19,18
True,19,23,24
